In [1]:
import pandas as pd
import re
import time
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from selenium import webdriver
from textacy.preprocess import preprocess_text
import nltk

from bs4 import BeautifulSoup as bs

In [2]:
start = time.time()

wd = webdriver.Chrome()

b_summary = []
b_detail = []
b_date = []
b_summary_clean = []
b_detail_clean = []

for i in range(1,5):
    print 'scrapping page : ', i
    url = 'https://www.singaporefoodie.com/blogs/page/' + str(i) + '/'    

    try:
        wd.get(url)
    except:
        continue
    response = wd.page_source
    soup = bs(response, 'html.parser')
    
    try:
        blogs = soup.find_all('h2', {'class' : 'entry-title'})
    except:
        continue
        
    for summary in blogs:
        try:
            href = summary.find('a').get('href')
            print 'href : ', href
        except:
            continue            
        
        try:
            wd.get(href)
        except:
            continue
        
        r = wd.page_source
        s = bs(r, 'html.parser')

        # find date
        try:
            date = s.find('span', {'class' : 'date'}).text
        except:
            continue
            
        # find detail
        try:
            y = s.find('div', {'class' : "entry-content"}).find_all('p')
        except:
            continue
        doc = []
        for p in y:
            if p.text:
                doc.append(p.text)
        d = ' '.join(doc)

        b_summary.append(summary.text)
        b_detail.append(d)
        b_date.append(date)
        
        # clean summary and detail 
        clean_summary = preprocess_text(summary.text, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        clean_detail = preprocess_text(d, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        b_summary_clean.append(clean_summary)
        b_detail_clean.append(clean_detail)
        
        print 'cleaned detail snippet : ', clean_detail[:75]
        
end = time.time()
exe_time = (end - start)/60
print 'Time taken :',(exe_time),' minutes'

print len(b_summary_clean)
print len(b_detail_clean)

scrapping page :  1
href :  https://www.singaporefoodie.com/goodbye-singapore-foodie/
cleaned detail snippet :  sorry for radio silence over the past month or so you may remember that sin
href :  https://www.singaporefoodie.com/10-places-long-lunch-weekend/
cleaned detail snippet :  nothing spells leisurely weekend like a long lunch i dont know about you bu
href :  https://www.singaporefoodie.com/checking-food-outlets-orchard-road/
cleaned detail snippet :  orchard road can be a bit of a minefield when it comes to food and drink op
href :  https://www.singaporefoodie.com/foc-sentosa/
cleaned detail snippet :  foc sentosa has given me a great excuse to write about sentosa after a long
href :  https://www.singaporefoodie.com/new-quayside-vibe/
cleaned detail snippet :  if you havent been to quayside at robertson quay for a while youre in for a
href :  https://www.singaporefoodie.com/kith-cafe-choose-from-8-locations/
cleaned detail snippet :  kith cafe is one of the first cafes i visited

cleaned detail snippet :  regular readers will know that i only write about a restaurant multiple tim
href :  https://www.singaporefoodie.com/tips-three-days-perth/
cleaned detail snippet :  ive been asked a lot recently about what id do with three days in perth sin
href :  https://www.singaporefoodie.com/5-brand-new-brunches/
cleaned detail snippet :  its always good to know about new brunches most of you have probably heard 
href :  https://www.singaporefoodie.com/cant-trust-singapore-food-blogs/
cleaned detail snippet :  ok its time to get a bit contentious one of the reasons i created singapore
href :  https://www.singaporefoodie.com/an-ang-mohs-guide-to-hawker-centres-in-singapore/
cleaned detail snippet :  an angmoh or gweilo is a slightly derogatory term used by singapoeans and o
href :  https://www.singaporefoodie.com/wet-markets-fresh-food-singapore/
cleaned detail snippet :  for the cheapest fresh food in singapore you cant go past wet markets the t
href :  https://www.singap

In [3]:
# remove stop words from cleaned summary and cleaned detail and lemmatize them

stoplist = set(stopwords.words('english') + \
               ['number',"singapore", "food", "im", "street", 
                'porn','watch', 'video', 'centre', '...',
                'eat', 'day', 'time', 'cdataadsbygoogle', 'windowadsbygoogle',
                'wa', 'ha', 'come', 'place', 'dish', 'bring', 'think', 'quite','located',
                'month', 'went', 'probably', 'am', 'pm', 'say', 'said','including','year','item',
                'youre', 'sure', 'dont', 'came','really', 'got', 'thing', 'address', 'photo',
                'credit', 'opening', 'hour'] \
                 + list(ENGLISH_STOP_WORDS))

In [4]:
start = time.time()

summary_tokens = []
for item in b_summary_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    summary_tokens.append(t)
    
detail_tokens = []
for item in b_detail_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    detail_tokens.append(t)   

    
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'
    
print len(summary_tokens)
print len(detail_tokens)

Time taken : 0.613649845123  seconds
80
80


In [5]:
start = time.time()

summary_lem =[]
wnl = WordNetLemmatizer()
for item in summary_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    summary_lem.append(t) 
    
detail_lem =[]
wnl = WordNetLemmatizer()
for item in detail_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    detail_lem.append(t)     
   
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'

print len(summary_lem)
print len(detail_lem)

Time taken : 5.22430300713  seconds
80
80


In [6]:
# create final list to fit in dataframe
flist = []
for i in range(len(b_summary)):
    f = [b_summary[i], b_detail[i], b_date[i], summary_lem[i], detail_lem[i]]
    flist.append(f)

In [7]:
df = pd.DataFrame(flist, columns = ['Title', 'Details', 'Date', 'Processed Summary', 'Processed Detail'])  
df.tail()

,Title,Details,Date,Processed Summary,Processed Detail
75,The Knolls for Best Outdoor Brunch,Ah that old chestnut of the “The Best Brunch ...,"November 12, 2014",knoll best outdoor brunch,ah old chestnut best brunch choice favourite b...
76,14 of the Best Things to do with Visitors in S...,The newest and shiniest Singapore sites often ...,"October 11, 2013",best thing visitor,newest shiniest site grab headline tourist goo...
77,Simple and Good Middle Eastern Food,Note that much of this post was written after ...,"November 3, 2015",simple good middle eastern,note post written visit old fillapita march re...
78,It’s goodbye from Singapore Foodie,Sorry for radio silence over the past month o...,"October 23, 2017",goodbye foodie,sorry radio silence past remember foodie hacke...
79,The Ultimate Guide to Brunch in Singapore,Brunch in Singapore has a reputation. If you’r...,"August 30, 2013",ultimate guide brunch,brunch reputation familiar citystate know famo...


In [8]:
len(df)

80

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
len(df)

49

In [11]:
x = df.iloc[3,1]
print x

FOC Sentosa has given me a great excuse to write about Sentosa after a long while. I go through love-hate phases with Sentosa. I mostly quite like it, there’s a holiday vibe, there’s the beach, and lots of options for things to do. Not that I’d live there or anything  The main issue though is the transport. Getting over there is usually not too bad, but getting stranded once you’re there is a nightmare. If you end up on the piece of road that only has the public bus, you can spend ages getting from A to B. FOC Sentosa poses no such problems. The location, at Palawan Beach, means you can get a taxi straight to the door. Which also means that if it’s tipping down with rain you can arrive relatively unscathed. Which was handy on our visit I can tell you. Seating at FOC Sentosa is split into two main sections. The main area has tables, mostly inside, with some out on a covered deck (that does get wet if it really rains). Next to that there is a bar area that looks out over the pool, and se

In [12]:
# pickle dataframe
df.to_pickle('singaporefoodie.pickle')

In [13]:
wd.quit()